In [1]:
from pycirclize_TEST import Circos
from plotly.offline import plot

# Initialize circos sectors
sectors = {"A": 10, "B": 15, "C": 12, "D": 20, "E": 15}
circos = Circos(sectors, space=5)

for sector in circos.sectors:
    # Plot sector axis & name text
    sector.axis(fillcolor=None, line=dict(color="black", dash="dashdot", width=2), opacity=0.5)
    sector.text(f"Sector: {sector.name}={sector.size}", font=dict(size=20))
    # Set Track01 (Radius: 75 - 100)
    track1 = sector.add_track((75, 100))
    track1.axis(fillcolor="tomato", opacity=0.5)
    track1.text(track1.name)
    # Set Track02 (Radius: 45 - 70)
    track2 = sector.add_track((45, 70))
    track2.axis(fillcolor="cyan", opacity=0.5)
    track2.text(track2.name)
    # Set Track03 (Radius: 15 - 40)
    track3 = sector.add_track((15, 40))
    track3.axis(fillcolor="lime", opacity=0.5)
    track3.text(track3.name)

fig = circos.plotfig()
plot(fig)

'temp-plot.html'

In [2]:
from pycirclize_TEST import Circos
from plotly.offline import plot

# Initialize circos sectors
sectors = {"A": 10, "B": 15, "C": 12, "D": 20, "E": 15}
circos = Circos(sectors, space=5, start=-270, end=30)

for sector in circos.sectors:
    # Plot sector axis & name text
    sector.axis(fillcolor="green", line=dict(color="red", dash="dashdot", width=10), opacity=1)
    sector.text(f"Sector: {sector.name}={sector.size}")

fig = circos.plotfig()
plot(fig)

'temp-plot.html'

In [3]:
from pycirclize_TEST import Circos
from plotly.offline import plot
import numpy as np
np.random.seed(0)

sectors = {"A": 10, "B": 15, "C": 12, "D": 20, "E": 15}
circos = Circos(sectors, space=5)
for sector in circos.sectors:
    # Plot sector name
    sector.text(f"Sector: {sector.name}", font=dict(size=20))
    # Create x positions & randomized y values for data plotting
    x = np.arange(sector.start, sector.end) + 0.5
    y = np.random.randint(0, 100, len(x))
    # Plot line
    line_track = sector.add_track((75, 100), r_pad_ratio=0.1)
    line_track.axis()
    line_track.xticks_by_interval(1)
    line_track.line(x, y)
    # # Plot points
    # points_track = sector.add_track((45, 70), r_pad_ratio=0.1)
    # points_track.axis()
    # points_track.scatter(x, y)
    # # Plot bar
    # bar_track = sector.add_track((15, 40), r_pad_ratio=0.1)
    # bar_track.axis()
    # bar_track.bar(x, y)

fig = circos.plotfig()
plot(fig)

NameError: name 'Normalize' is not defined

In [1]:
from svgpathtools import Arc, Line, Path
import numpy as np
from math import cos, sin
import cmath

class ArcRectangle(Path):
    """ArcRectangle shape based on polar coordinates, as an SVG Path."""

    def __init__(self, radr: tuple[float, float], width: float, height: float, step: float = 0.02):
        """
        Parameters
        ----------
        radr : tuple[float, float]
            Anchor point (rad, r), starting angle and radius.
        width : float
            Radian width of the arc rectangle.
        height : float
            Radial height of the arc rectangle.
        step : float
            Step in radians to sample arcs (optional).
        """
        self.radr = radr
        self.width = width
        self.height = height
        self.step = step

        min_rad, min_r = radr
        max_rad = min_rad + width
        max_r = min_r + height

        arc_rads = np.arange(min_rad, max_rad, step)
        arc_rads = np.append(arc_rads, max_rad)

        # Bottom and top arc points
        bottom_arc_pts = [self._polar_to_complex(theta, min_r) for theta in arc_rads]
        top_arc_pts = [self._polar_to_complex(theta, max_r) for theta in arc_rads[::-1]]

        # Construct path segments
        segments = []

        # Bottom arc as small arcs between each point
        for p0, p1 in zip(bottom_arc_pts[:-1], bottom_arc_pts[1:]):
            arc = Arc(start=p0, end=p1, radius=min_r + 0j,
                      rotation=0, large_arc=False, sweep=True)
            segments.append(arc)

        # Vertical side
        segments.append(Line(bottom_arc_pts[-1], top_arc_pts[0]))

        # Top arc
        for p0, p1 in zip(top_arc_pts[:-1], top_arc_pts[1:]):
            arc = Arc(start=p0, end=p1, radius=max_r + 0j,
                      rotation=0, large_arc=False, sweep=False)
            segments.append(arc)

        # Vertical closing line
        segments.append(Line(top_arc_pts[-1], bottom_arc_pts[0]))

        # Initialize as a Path
        super().__init__(*segments)

    def _polar_to_complex(self, theta: float, r: float) -> complex:
        """Convert polar coordinates to complex (x + iy)."""
        return r * cmath.exp(1j * theta)

In [1]:
import numpy as np

lol = 0.01

class PolarSVGPathBuilder:
    def __init__(self, step: float = 0.01):
        self.step = step

    @staticmethod
    def _polar_to_cart(theta: float, r: float) -> tuple[float, float]:
        return r * np.cos(theta), r * np.sin(theta)

    @staticmethod
    def _svg_path_from_points(points: list[tuple[float, float]], closed: bool = False) -> str:
        if not points:
            return ""
        path = [f"M {points[0][0]} {points[0][1]}"]
        path += [f"L {x} {y}" for x, y in points[1:]]
        if closed:
            path.append("Z")
        return " ".join(path)
    
    def arc_rectangle(self, radr: tuple[float, float], width: float, height: float) -> str:
        min_rad, min_r = radr
        max_rad = min_rad + width
        max_r = min_r + height

        arc_rads = np.arange(min_rad, max_rad, self.step)
        arc_rads = np.append(arc_rads, max_rad)

        bottom_arc = [PolarSVGPathBuilder._polar_to_cart(theta, min_r) for theta in arc_rads]
        upper_arc = [PolarSVGPathBuilder._polar_to_cart(theta, max_r) for theta in arc_rads[::-1]]
        points = bottom_arc + upper_arc + [bottom_arc[0]]
        return PolarSVGPathBuilder._svg_path_from_points(points, closed=True)
    
    def arc_line(self, rad_lim: tuple[float, float], r_lim: tuple[float, float]) -> str:
        rad_start, rad_end = rad_lim
        r_start, r_end = r_lim

        if rad_start > rad_end:
            rad_start, rad_end = rad_end, rad_start
            r_start, r_end = r_end, r_start

        arc_rads = np.arange(rad_start, rad_end, self.step)
        arc_rads = np.append(arc_rads, rad_end)
        arc_rs = np.linspace(r_start, r_end, len(arc_rads))

        points = [PolarSVGPathBuilder._polar_to_cart(t, r) for t, r in zip(arc_rads, arc_rs)]
        return PolarSVGPathBuilder._svg_path_from_points(points)
    
    # def line(self, rad_lim: tuple[float, float], r_lim: tuple[float, float]) -> str:
    #     points = [PolarSVGPathBuilder._polar_to_cart(rad, r) for rad, r in zip(rad_lim, r_lim)]
    #     return PolarSVGPathBuilder._svg_path_from_points(points)
    
    def arc_arrow(
        self,
        rad: float,
        r: float,
        drad: float,
        dr: float,
        head_length: float = np.pi / 90,
        shaft_ratio: float = 0.5,
    ) -> str:
        shaft_size = dr * shaft_ratio
        y_shaft_bottom = r + ((dr - shaft_size) / 2)
        y_shaft_upper = r + dr - ((dr - shaft_size) / 2)

        is_forward = drad >= 0
        drad = abs(drad)
        head_length = min(head_length, drad)

        rad_shaft_tip = rad + (drad - head_length) if is_forward else rad - (drad - head_length)
        rad_arrow_tip = rad + drad if is_forward else rad - drad

        p1 = rad, y_shaft_bottom
        p2 = rad_shaft_tip, y_shaft_bottom
        p3 = rad_shaft_tip, r
        p4 = rad_arrow_tip, (r + r + dr) / 2
        p5 = rad_shaft_tip, r + dr
        p6 = rad_shaft_tip, y_shaft_upper
        p7 = rad, y_shaft_upper

        shaft_rads = np.arange(p1[0], p2[0], self.step if is_forward else -self.step)
        bottom_shaft = [PolarSVGPathBuilder._polar_to_cart(t, p1[1]) for t in shaft_rads]
        top_shaft = [PolarSVGPathBuilder._polar_to_cart(t, p7[1]) for t in shaft_rads[::-1]]
        head = [PolarSVGPathBuilder._polar_to_cart(*pt) for pt in [p2, p3, p4, p5, p6]]
        path_points = bottom_shaft + head + top_shaft + [PolarSVGPathBuilder._polar_to_cart(*p1)]

        return PolarSVGPathBuilder._svg_path_from_points(path_points, closed=True)

In [6]:
import plotly.graph_objects as go

# Initialize Plotly figure
fig = go.Figure()

svg_path = PolarSVGPathBuilder().arc_rectangle(radr=(0,0), width=2, height=10)

# Add the shape to the layout
fig.update_layout(
    shapes=[
        dict(
            type="path",
            path=svg_path,
            fillcolor="lightblue",
            line=dict(color="blue", width=2)
        )
    ],
    xaxis=dict(range=[-10, 10], zeroline=False),
    yaxis=dict(range=[-10, 10], scaleanchor="x", scaleratio=1, zeroline=False),
    showlegend=False,
)

In [ ]:
import pycirclize_TEST

In [ ]:
from pycirclize_TEST import Circos
from pycirclize_TEST.utils import load_eukaryote_example_dataset

# Load hg38 dataset (https://github.com/moshi4/pycirclize-data/tree/main/eukaryote/hg38)
chr_bed_file, cytoband_file, _ = load_eukaryote_example_dataset("hg38")

# Initialize Circos from BED chromosomes
circos = Circos.initialize_from_bed(chr_bed_file, space=3)
circos.text("Homo sapiens (hg38)", size=15)

# # Add cytoband tracks from cytoband file
# circos.add_cytoband_tracks((95, 100), cytoband_file)

# Plot chromosome name
for sector in circos.sectors:
    sector.text(sector.name, size=10)

fig = circos.plotfig()

In [ ]:
chr_bed_file

In [ ]:
cytoband_file